### 0 - Bibliotecas

In [1]:
## Processamento de dados
import numpy as np
import pandas as pd
import pandas_profiling
from pandas_profiling import ProfileReport
import random

#Bibliotecas utilizadas
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

## Vusualização de dados
import matplotlib.pyplot as plt
from IPython.display import Image

### pip install --upgrade scikit-learn
%matplotlib inline

In [2]:
import warnings 
warnings.filterwarnings("ignore")

### 1 - Dado Original

In [3]:
# Leitura do dataset
df = pd.read_csv('forestfires.csv')
# Transformação da dimensão 'área' pelo seu logaritmo (depois de somar 1 para evitar zeros)
df['Logarea']=np.log10(df['area']+1)
# Codificação de rótulos para dimensões categóricas (dia e mês)
# Codificação MES
enc = LabelEncoder()
enc.fit(df['month'])
enc.classes_
df['month_encoded']=enc.transform(df['month'])
# Codificação DIA
enc.fit(df['day'])
enc.classes_
df['day_encoded']=enc.transform(df['day'])
df

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,Logarea,month_encoded,day_encoded
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00,0.000000,7,0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00,0.000000,10,5
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00,0.000000,10,2
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00,0.000000,7,0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00,0.000000,7,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,4,3,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44,0.871573,1,3
513,2,4,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29,1.742647,1,3
514,7,4,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16,1.084934,1,3
515,1,4,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00,0.000000,1,2


#### OBSERVAÇÃO - 517 ELEMENTOS - FAREMOS O SAMPLING DE 50 ELEMENTOS

### 2 - Amostragem Ponderada (AP)

In [4]:
def get_weighted_sample(df,n):
    def get_class_prob(x):
        weight_x = int(np.rint(n * len(x[x.Logarea != 0]) / len(df[df.Logarea != 0])))
        sampled_x = x.sample(weight_x).reset_index(drop=True)
        return (sampled_x)
        # we are grouping by the target class we use for the proportions

    weighted_sample = df.groupby('rain').apply(get_class_prob)
    print(weighted_sample["rain"].value_counts())
    return (weighted_sample)
AP = get_weighted_sample(df,50)
#sample
AP.describe()

0.0    50
Name: rain, dtype: int64


,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,Logarea,month_encoded,day_encoded
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.00000,50.0,50.000000,50.000000,50.000000,50.000000
mean,4.080000,4.100000,90.634000,113.378000,590.186000,8.380000,19.238000,46.220000,3.85600,0.0,13.336400,0.571152,5.820000,2.460000
std,2.354674,1.216385,3.040583,56.436145,205.212066,3.946375,6.108605,18.313918,1.72735,0.0,39.663664,0.613278,4.516139,1.929471
min,1.000000,2.000000,81.600000,18.200000,55.000000,1.900000,2.200000,19.000000,1.30000,0.0,0.000000,0.000000,1.000000,0.000000
25%,2.000000,4.000000,90.125000,83.200000,562.575000,6.475000,17.150000,35.000000,2.20000,0.0,0.000000,0.000000,1.000000,1.000000
50%,4.000000,4.000000,91.600000,105.000000,665.600000,7.800000,19.750000,42.000000,4.00000,0.0,1.855000,0.454371,5.000000,2.000000
75%,6.000000,5.000000,92.200000,138.425000,719.625000,9.150000,22.725000,53.000000,5.27500,0.0,8.007500,0.954154,11.000000,3.750000
max,9.000000,8.000000,95.000000,290.000000,855.300000,21.300000,30.600000,96.000000,8.50000,0.0,212.880000,2.330170,11.000000,6.000000


### 3 - APLICAÇÃO DOS 4 MÉTODOS
    - a) support vector regression
    - b) decision tree
    - c) random forest
    - d) 3-layer dense neural network

In [6]:
### Método 1 - Support Vector Regressor (SVR) - Regressor vetorial de suporte (SVR)
# configuração do tamanho do conjunto-teste
test_size=0.4
df_sample = AP
# segmentação teste-treino
X_data=df_sample.drop(['area','Logarea','month','day'],axis=1)
y_data=df_sample['Logarea']
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=test_size)
y_train=y_train.values.reshape(y_train.size,1)
#y_train=y_train.reshape(y_train.size,1)
def rec(m,n,tol):
    if type(m)!='numpy.ndarray':
        m=np.array(m)
    if type(n)!='numpy.ndarray':
        n=np.array(n)
    l=m.size
    percent = 0
    for i in range(l):
        if np.abs(10**m[i]-10**n[i])<=tol:
            percent+=1
    return 100*(percent/l)
# Definir o limite máximo de tolerância para o eixo x da curva REC - Para este problema, isso 
# representa o valor absoluto do erro na previsão do resultado, ou seja, a área queimada
tol_max=20
### Support Vector Regressor (SVR)
scaler = StandardScaler()
# Parameter grid for the Grid Search
param_grid = {'C': [0.01,0.1,1, 10], 'epsilon': [10,1,0.1,0.01,0.001,0.0001], 'kernel': ['rbf']}
grid_SVR = GridSearchCV(SVR(),param_grid,refit=True,verbose=0,cv=5)
grid_SVR.fit(scaler.fit_transform(X_train),scaler.fit_transform(y_train))
print("Best parameters obtained by Grid Search:",grid_SVR.best_params_)
a=grid_SVR.predict(X_test)
RMSE_SVR = np.sqrt(np.mean((y_test-a)**2))
#print("RMSE for Support Vector Regression:",np.sqrt(np.mean((y_test-a)**2)))
print("RMSE for Support Vector Regression:",RMSE_SVR)
rec_SVR=[]
for i in range(tol_max):
    rec_SVR.append(rec(a,y_test,i))
# Método 2 - Decision Tree
tree_model = DecisionTreeRegressor(max_depth=10,criterion='mae')
tree_model.fit(scaler.fit_transform(X_train),scaler.fit_transform(y_train))
a=tree_model.predict(X_test)
RMSE_DT = np.sqrt(np.mean((y_test-a)**2))
#print("RMSE for Decision Tree:",np.sqrt(np.mean((y_test-a)**2)))
print("RMSE for Decision Tree:",RMSE_DT)
rec_DT=[]
for i in range(tol_max):
    rec_DT.append(rec(a,y_test,i))
# Método 3 - Random Forest
from sklearn.ensemble import RandomForestRegressor
param_grid = {'max_depth': [5,10,15,20,50], 'max_leaf_nodes': [2,5,10], 'min_samples_leaf': [2,5,10],
             'min_samples_split':[2,5,10]}
grid_RF = GridSearchCV(RandomForestRegressor(),param_grid,refit=True,verbose=0,cv=5)
grid_RF.fit(X_train,y_train)
print("Best parameters obtained by Grid Search:",grid_RF.best_params_)
a=grid_RF.predict(X_test)
RMSE_RF=np.sqrt(np.mean((y_test-a)**2))
print("RMSE for Random Forest:",RMSE_RF)
rec_RF=[]
for i in range(tol_max):
    rec_RF.append(rec(a,y_test,i))
# Método 4 - Layer Dense Neural Network - Deep network ( Keras (TensorFlow))
from keras.models import Sequential
import keras.optimizers as opti
from keras.layers import Dense, Activation,Dropout
# layers - camadas
model = Sequential()
model.add(Dense(100, input_dim=12))
model.add(Activation('selu'))
model.add(Dropout(0.3))
model.add(Dense(100))
model.add(Dropout(0.3))
model.add(Activation('selu'))
model.add(Dense(50))
model.add(Activation('elu'))
model.add(Dense(1))
model.summary()
# taxa de aprendizado e otimização
learning_rate=0.001
optimizer = opti.RMSprop(lr=learning_rate)
model.compile(optimizer=optimizer,loss='mse')
#### Input data and mode fitting
data=X_train
target = y_train
model.fit(data, target, epochs=100, batch_size=0 ,verbose=0)
#### Prediction and RMSE
a=model.predict(X_test)
RMSE_LDNN = np.sqrt(np.mean((y_test-a.reshape(a.size,))**2))
#print("RMSE for Deep Network:",np.sqrt(np.mean((y_test-a.reshape(a.size,))**2)))
print("RMSE for Deep Network:",RMSE_LDNN)
rec_NN=[]
for i in range(tol_max):
    rec_NN.append(rec(a,y_test,i))

Best parameters obtained by Grid Search: {'C': 1, 'epsilon': 1, 'kernel': 'rbf'}
RMSE for Support Vector Regression: 0.45279350832087834
RMSE for Decision Tree: 1.4890460271213795
Best parameters obtained by Grid Search: {'max_depth': 15, 'max_leaf_nodes': 5, 'min_samples_leaf': 2, 'min_samples_split': 5}
RMSE for Random Forest: 0.6366186521514537
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 100)               1300      
                                                                 
 activation_3 (Activation)   (None, 100)               0         
                                                                 
 dropout_2 (Dropout)         (None, 100)               0         
                                                                 
 dense_5 (Dense)             (None, 100)               10100     
                                  

,Support Vector Regression,Decision Tree,Random Forest,Deep Network
RMSE,0.452794,1.489046,0.636619,1.461701


In [7]:
Ranking_ForrestFire = pd.DataFrame()
index_RFF = ['RMSE']
Ranking_ForrestFire  = pd.DataFrame(index=index_RFF)
Ranking_ForrestFire  ['Support Vector Regression'] = RMSE_SVR
Ranking_ForrestFire  ['Decision Tree'] = RMSE_DT
Ranking_ForrestFire  ['Random Forest'] = RMSE_RF
Ranking_ForrestFire  ['Deep Network'] = RMSE_LDNN
#
plt.figure(figsize=(10,6))
plt.title("Curvas REC\n",fontsize=20)
plt.xlabel("Erro Absoluto (tolerancia) na predição ($ha$)",fontsize=15)
plt.ylabel("Porcentagem da predição correta ",fontsize=15)
plt.xticks([i for i in range(0,tol_max+1,1)],fontsize=13)
plt.ylim(-10,100)
plt.xlim(-2,tol_max)
plt.yticks([i*20 for i in range(6)],fontsize=18)
plt.grid(True)
plt.plot(range(tol_max),rec_SVR,'--',lw=3)
plt.plot(range(tol_max),rec_DT,'*-',lw=3)
plt.plot(range(tol_max),rec_RF,'o-',lw=3)
plt.plot(range(tol_max),rec_NN,'k-',lw=3)
plt.legend(['SVR','Decision Tree','Random Forest','Deep NN'],fontsize=13)
Ranking_ForrestFire 